## Enpoint 1:
- def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:
- Año	Cantidad de Items	Contenido Free
- 2023	50	27%
- 2022	45	25%
- xxxx	xx	xx%

In [1]:
import pandas as pd

# Leo el archivo que voy a utilizar para este endpoint
games = pd.read_parquet('../Dataset/games_clean.parquet')


In [2]:
games.head(10)

,id,title,developer,release_date,genres,tags,price
0,761140,Lost Summoner Kitty,Kotoshiro,2018-01-04,"Action, Casual, Indie, Simulation, Strategy","Strategy, Action, Indie, Casual, Simulation",4.99
1,643980,Ironbound,Secret Level SRL,2018-01-04,"Free to Play, Indie, RPG, Strategy","Free to Play, Strategy, Indie, RPG, Card Game,...",0.00
2,670290,Real Pool 3D - Poolians,Poolians.com,2017-07-24,"Casual, Free to Play, Indie, Simulation, Sports","Free to Play, Simulation, Sports, Casual, Indi...",0.00
3,767400,弹炸人2222,彼岸领域,2017-12-07,"Action, Adventure, Casual","Action, Adventure, Casual",0.99
4,772540,Battle Royale Trainer,Trickjump Games Ltd,2018-01-04,"Action, Adventure, Simulation","Action, Adventure, Simulation, FPS, Shooter, T...",3.99
5,774277,SNOW - All Access Pro Pass,Poppermost Productions,2018-01-04,"Free to Play, Indie, Simulation, Sports","Free to Play, Indie, Simulation, Sports",18.99
6,774278,SNOW - All Access Legend Pass,Poppermost Productions,2018-01-04,"Free to Play, Indie, Simulation, Sports","Free to Play, Indie, Simulation, Sports",29.99
7,770380,Army of Tentacles: (Not) A Cthulhu Dating Sim:...,Stegalosaurus Game Development,2018-01-04,"Action, Adventure, Casual, Indie, RPG","Action, Adventure, RPG, Indie, Casual",10.99
8,768880,Beach Rules,Copperpick Studio,2018-01-04,"Casual, Indie","Casual, Indie, Pixel Graphics, Cute, 2D",3.99
9,765320,Planetarium 2 - Zen Odyssey,Ghulam Jewel,2018-01-03,"Casual, Indie, Simulation","Indie, Casual, Simulation",2.99


In [3]:
games = games[['developer', 'release_date', 'price']]
games.head(5)

,developer,release_date,price
0,Kotoshiro,2018-01-04,4.99
1,Secret Level SRL,2018-01-04,0.00
2,Poolians.com,2017-07-24,0.00
3,彼岸领域,2017-12-07,0.99
4,Trickjump Games Ltd,2018-01-04,3.99


In [4]:
#Creamos una columna de contenido free representado por 1 y no free por 0
games['free'] = games['price'].apply(lambda x: 1 if x == 0 else 0)

#Despues elimino la columna "price"
games.drop('price', axis=1, inplace=True)

# Revisamos los cambios
games.head(5)

,developer,release_date,free
0,Kotoshiro,2018-01-04,0
1,Secret Level SRL,2018-01-04,1
2,Poolians.com,2017-07-24,1
3,彼岸领域,2017-12-07,0
4,Trickjump Games Ltd,2018-01-04,0


In [ ]:
#Exporto el archivo
games.to_parquet('../Dataset/developer.parquet', compression='snappy', index=False)

-------------------------------------------------------------------------------------------------------------------------

### Probando endpoint

In [11]:
def developer(developer : str):
    """
    End point 1
    - def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free
      por año según empresa desarrolladora. Ejemplo de retorno:

    - Año	Cantidad de Items	Contenido Free
    - 2023	      50	              27%
    - 2022	      45	              25%
    - xxxx	      xx	              xx%
    Bohemia Interactive

    """
    #games = pd.read_parquet('Dataset/developer.parquet')
    global games
    # Filtramos por el desarrollador ingresado
    games = games[games['developer'] == developer]

    # Agrupamos por año y contamos la cantidad de items
    games['Año'] = games['release_date'].dt.year
    games_year = games.groupby('Año').size().reset_index(name='Cantidad de Items')

    # Agrupamos por año y sumamos la cantidad de items free
    games_free = games.groupby('Año')['free'].sum().reset_index(name='free')

    # Hacemos un merge de ambos dataframes mediante la columna Año
    games = pd.merge(games_year, games_free, on='Año')

    # Calculamos el porcentaje de contenido free
    games['Contenido Free'] = round(games['free'] / games['Cantidad de Items'] * 100, 2)

    # Eliminamos la columna free
    games.drop(columns=['free'], inplace=True)

    # Convertimos el dataframe a un diccionario
    games = games.to_dict('records')
    
    return games

prueba = developer('Bohemia Interactive')
prueba

[{'Año': 2000, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2007, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2009, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2010, 'Cantidad de Items': 3, 'Contenido Free': 0.0},
 {'Año': 2011, 'Cantidad de Items': 3, 'Contenido Free': 0.0},
 {'Año': 2012, 'Cantidad de Items': 4, 'Contenido Free': 0.0},
 {'Año': 2013, 'Cantidad de Items': 4, 'Contenido Free': 0.0},
 {'Año': 2014, 'Cantidad de Items': 4, 'Contenido Free': 0.0},
 {'Año': 2015, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2016, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2017, 'Cantidad de Items': 6, 'Contenido Free': 0.0}]

-------------------------------------------------------------------------------------------------------------------------

## Enpoint 4:
- def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
- Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [3]:
import pandas as pd
games = pd.read_parquet('../Dataset/games_clean.parquet')
reviews = pd.read_parquet('../Dataset/reviews_clean.parquet')

In [4]:
games.columns

Index(['id', 'title', 'developer', 'release_date', 'genres', 'tags', 'price'], dtype='object')

In [5]:
reviews.columns

Index(['item_id', 'negativo', 'neutral', 'positivo'], dtype='object')

In [6]:
#Creamos dos dataframes donde contengan las columnas que vamos a utilizar 
games = games[['developer', 'title', 'id', 'genres', 'release_date']]
reviews = reviews[['item_id', 'positivo']]

In [19]:
games.head(5)

,developer,title,id,genres,release_date,release_year
46,Funcom,The Longest Journey,6310,"Action, Adventure, RPG",2000-11-17,2000
98,"Epic Games, Inc.",Unreal Tournament: Game of the Year Edition,13240,Action,2000-10-25,2000
451,Egosoft,X: Tension,2850,"Simulation, Strategy",2000-06-01,2000
1840,Bohemia Interactive,"Fairy Tale About Father Frost, Ivan and Nastya",289320,"Adventure, Casual",2000-12-20,2000
1896,Humongous Entertainment,Pajama Sam 3: You Are What You Eat From Your H...,292800,"Adventure, Casual",2000-04-01,2000


In [8]:
reviews.head(5)

,item_id,positivo
0,10,51
1,10090,47
2,10130,2
3,10140,0
4,10150,9


In [9]:
#Combertimos el archivo a parquet
games.to_parquet('../Dataset/best_developer_games.parquet', compression='snappy', index=False)
reviews.to_parquet('../Dataset/best_developer_reviews.parquet', compression='snappy', index=False)

-------------------------------------------------------------------------------------------------------------------------

### Probando endpoint

In [16]:
def best_developer_year(año: int):
    """
    4) enpoint:
    - def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
    - Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
    """
    global games, reviews
    games['release_year'] = games['release_date'].dt.year
    games_filtered = games[games['release_year'] == año]
    filtered_reviews = games_filtered.merge(reviews, left_on='id', right_on='item_id', how='inner')
    filtered_reviews = filtered_reviews[['developer', 'title', 'positivo']]
    filtered_reviews = filtered_reviews.groupby(['developer']).agg({'positivo': 'sum'}).reset_index()
    filtered_reviews = filtered_reviews.sort_values(by='positivo', ascending=False)

    top_developers = filtered_reviews.head(3)['developer'].tolist()
    return [
        {"Puesto 1": top_developers[0]},
        {"Puesto 2": top_developers[1]},
        {"Puesto 3": top_developers[2]}
    ]

prueba = best_developer_year(2000)
print(prueba)

[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Ion Storm'}, {'Puesto 3': 'Rebellion'}]


-------------------------------------------------------------------------------------------------------------------------

## Enpoint 5:
- def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
- Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [12]:
import pandas as pd
games = pd.read_parquet('../Dataset/games_clean.parquet')
reviews = pd.read_parquet('../Dataset/reviews_clean.parquet')

In [13]:
reviews.columns

Index(['item_id', 'negativo', 'neutral', 'positivo'], dtype='object')

In [14]:
#Creamos dos dataframes donde contengan las columnas que vamos a utilizar 
games = games[['developer','id']]
reviews = reviews[['item_id', 'positivo', 'negativo']]

In [15]:
games.head()

,developer,id
0,Kotoshiro,761140
1,Secret Level SRL,643980
2,Poolians.com,670290
3,彼岸领域,767400
4,Trickjump Games Ltd,772540


In [16]:
reviews.head()

,item_id,positivo,negativo
0,10,51,6
1,10090,47,5
2,10130,2,0
3,10140,0,1
4,10150,9,0


In [26]:
#Combertimos el archivo a parquet
games.to_parquet('../Dataset/developer_review_games.parquet', compression='snappy', index=False)
reviews.to_parquet('../Dataset/developer_review_reviews.parquet', compression='snappy', index=False)

-------------------------------------------------------------------------------------------------------------------------

### Probando endpoint

In [17]:
def developer_reviews_analysis(desarrolladora: str ):
    """
    Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave
    y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados
    con un análisis de sentimiento como valor positivo o negativo.

    Ejemplo de retorno: {'Valve': {'Negativas': 182, 'Positivas': 278}}
    """
    #games = pd.read_parquet('Dataset/developer_review_games.parquet')  # Cargar datos de juegos
    #reviews = pd.read_parquet('Dataset/developer_review_reviews.parquet')  # Cargar datos de reseñas
    global games, reviews

    # Filtrar los juegos por el desarrollador especificado
    developer_games = games[games['developer'] == desarrolladora]
    
    # Combinar datos de juegos y reseñas basado en el id del juego
    merged_data = developer_games.merge(reviews, left_on='id', right_on='item_id', how='inner')

    # Verificar si no hay datos disponibles para el desarrollador
    if merged_data.empty:
        return {desarrolladora: {'Negativas': 0, 'Positivas': 0}}

    # Calcular el total de reseñas negativas y positivas
    total_negativas = merged_data['negativo'].sum()
    total_positivas = merged_data['positivo'].sum()

    # Retornar el resultado en el formato esperado
    return {desarrolladora: {'Negativas': int(total_negativas), 'Positivas': int(total_positivas)}}

prueba3 = developer_reviews_analysis("Bohemia Interactive")
prueba3

{'Bohemia Interactive': {'Negativas': 293, 'Positivas': 868}}